In [ ]:
#ransomhouse

def filter_recent_actions_from_json(json_data):
    """Filtra as entradas com actionDate dentro dos últimos 7 dias."""

    # Definindo os últimos 7 dias
    hoje = datetime.now()
    dias_7_atras = hoje - timedelta(days=7)

    def is_within_last_7_days(date_str):
        """Verifica se a data está nos últimos 7 dias."""
        # Verificar se a string não está vazia
        if not date_str:
            return False
        
        # Converter a string da data para um objeto datetime
        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%Y')
            # Verificar se a data está dentro do intervalo
            return dias_7_atras <= date_obj <= hoje
        except ValueError:
            return False

    filtered_data = []
    
    for entry in json_data.get('data', []):
        # Verifica se a chave 'actionDate' existe e se a data está nos últimos 7 dias
        if 'actionDate' in entry and is_within_last_7_days(entry['actionDate']):
            filtered_data.append({
                'header': entry.get('header', ''),
                'url': entry.get('url', ''),
                'actionDate': entry.get('actionDate', '')
            })

    return filtered_data

# Exemplo de uso
if __name__ == "__main__":
    # Exemplo de JSON
    json_example = json.loads(df_to_analize['html'][0])

    # Filtrando os dados
    filtered_data = filter_recent_actions_from_json(json_example)

    # Exibindo o resultado
    print(json.dumps(filtered_data, indent=4))


In [ ]:
#monti

dates_to_find = df_to_analize['datas_D-7'][1]
html_content = df_to_analize['html'][1]
def find_titles_by_dates(html_content, dates_to_find):
    
    # Criando um objeto BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Inicializando a lista de resultados
    results = []

    # Iterando na lista de datas e coletando títulos
    for date in dates_to_find:
        # Encontrando a div que contém a data específica
        div = soup.find('div', class_='col-auto published', string=lambda text: text and date in text)
        
        if div:
            # Encontrando o pai da div para obter o título
            h5 = div.find_previous('h5')
            title = h5.text.strip() if h5 else 'Título não encontrado'
            results.append({'title': title, 'date': div.text.strip()})

    return results

# Chamando a função e exibindo os resultados
results = find_titles_by_dates(html_content, dates_to_find)
for result in results:
    print(f"Título: {result['title']}, Data: {result['date']}")

In [ ]:
# play

html_content = df_to_analize['html'][2]

def collect_recent_publications(html_content):
    # Criando um objeto BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Data atual e data de corte para 7 dias atrás
    current_date = datetime.now()
    cutoff_date = current_date - timedelta(days=7)

    # Coletando os dados
    data_rows = []

    for th in soup.find_all('th', class_='News'):
        # Título é o texto do th, excluindo as tags filhas
        title = th.contents[0].strip()  # Apenas o texto do <th> antes do <div>

        # Encontrando o link
        link_div = th.find('i', class_='link')
        link = link_div.next_sibling.strip() if link_div and link_div.next_sibling else None
        
        # Encontrando a data de publicação
        publication_date_div = th.find(string=lambda text: text and 'publication date:' in text)
        publication_date = publication_date_div.strip().split(':')[-1].strip() if publication_date_div else None
        
        # Verificando se a data de publicação está nos últimos 7 dias
        if publication_date:
            pub_date = datetime.strptime(publication_date, '%Y-%m-%d')
            if pub_date >= cutoff_date:  # Apenas incluir se a data de publicação for maior ou igual à data de corte
                data_rows.append({'title': title, 'link': link, 'publication_date': publication_date})

    return data_rows

# Chamando a função e exibindo os resultados
recent_publications = collect_recent_publications(html_content)
for data in recent_publications:
    print(f"Título: {data['title']}, Link: {data['link']}, Data de Publicação: {data['publication_date']}")

In [ ]:
#flocker

html_content = df_to_analize['html'][3]
dates_to_find = df_to_analize['datas_D-7'][3]

def extract_data_from_html(html_content, dates_to_find):
    
    soup = BeautifulSoup(html_content, 'html.parser')
    results = []

    # Iterando sobre os artigos
    for article in soup.find_all('article'):
        # Coletando a data de publicação
        publication_date_span = article.find('span', itemprop='datePublished')
        publication_date = publication_date_span.text.strip() if publication_date_span else None
        
        # Verificando se a data está na lista de datas a serem procuradas
        if publication_date in dates_to_find:
            # Coletando o título
            title_tag = article.find('h2', itemprop='headline')
            title = title_tag.text.strip() if title_tag else 'Título não encontrado'
            
            # Coletando a descrição e extraindo o site
            description_tag = article.find('div', class_='ast-excerpt-container')
            description = description_tag.text.strip() if description_tag else ''
            
            # Usando expressão regular para encontrar o site na descrição
            site_match = re.search(r'\b(?:[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}|(?:http|https):\/\/[^\s]+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})\b', description)
            site = site_match.group(0) if site_match else 'Site não encontrado'
            
            # Adicionando o resultado à lista
            results.append({
                'title': title,
                'publication_date': publication_date,
                'site': site
            })

    return results

# Chamando a função e exibindo os resultados
results = extract_data_from_html(html_content, dates_to_find)
for result in results:
    print(f"Título: {result['title']}, Data: {result['publication_date']}, Site: {result['site']}")

In [ ]:
#handala

html_content = df_to_analize['html'][4]
target_dates  = df_to_analize['datas_D-7'][4]

def extract_info_from_html(html_content, target_dates):
    # Fazendo o parsing do HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Coletando informações
    results = []
    for li in soup.find_all('li', class_='wp-block-post'):
        title = li.find('h2').find('a').text
        date = li.find('time').get('datetime').split('T')[0]  # pegando apenas a data
        description = li.find('p', class_='wp-block-post-excerpt__excerpt').text

        # Verificando se a data está na lista de datas
        if date in target_dates:
            # Extraindo o site da descrição
            site_match = re.search(r'\b(?:https?://|www\.)?([\w.-]+(?:\.[a-z]{2,}))\b', description)
            site = site_match.group(0) if site_match else None
            
            # Adicionando os resultados
            results.append({
                'title': title,
                'date': date,
                'site': site
            })

    return results

# Chamando a função e imprimindo os resultados
results = extract_info_from_html(html_content, target_dates)
for result in results:
    print(result)